# UCA Rules Engine Notebook

The **UCA Rules Engine** lets you describe *when* certain parts of your pipeline should run.  
Instead of manually kicking off tasks, you define **rules** that connect conditions to actions.  

For example:  
- **When** a dataset folder reaches 10,000 images -> **then** trigger labeling and training services.  
- **When** a model’s accuracy drops below a threshold -> **then** start a fine-tuning job.  
- **When** new data arrives in a watched location -> **then** send it for preprocessing.  

In short, rules help you automate *when* to run key steps (like labeling, training, or fine-tuning models) so workflows stay consistent and adaptive without constant manual intervention.  

### 1) Install dependencies

Before using the Rules Engine client, we need to install a few Python dependencies and pull in the **UCA repo** (which contains the client code). 

In [22]:
!pip install -qU "jsonschema>=4.20,<5" "requests>=2.31,<3" tapipy python-dotenv
!pip install -q git+https://github.com/ICICLE-ai/UCA.git


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


**Importing packages**

In [1]:
from getpass import getpass
from pprint import pprint

from tapipy.tapis import Tapis

from database.rules_engine.rules_engine import RulesEngine

### 2) Tapis authentication

- Enter Tapis tenant, username, and password in the cell.
- The notebook fetches an access token.

In [2]:
# Tapis login (client-side only)
TAPIS_URL = "https://tacc.tapis.io"   # or your tenant base
USERNAME  = input("username : ")
PASSWORD  = getpass("password : ")

t = Tapis(base_url=TAPIS_URL, username=USERNAME, password=PASSWORD)
t.get_tokens()
ACCESS_TOKEN = t.access_token.access_token
print("Access token acquired (length):", len(ACCESS_TOKEN))

username :  molakalmuru
password :  ········


Access token acquired (length): 912


### 3) Connect the client

Creating a `RulesEngine` client using your Tapis access token.  
This safely signs you in so you can run the next steps (health check, create/list/update/delete rules).

In [3]:
re = RulesEngine(tapis_token=ACCESS_TOKEN)

### 4) Check server health

Before creating any rules, let’s confirm the Rules Engine service is up and reachable.  
This call returns a simple status message so you know the backend is ready to handle requests.

Expect: `{ "status": "ok" }`

In [4]:
pprint(re.health())

{'status': 'ok'}


### 5) Create a new rule  

Here we define a rule and send it to the Rules Engine.  

- A rule is essentially a **trigger** defined by the user.  
- For example, it can trigger labeling or training apps when the folder reaches a certain number of images, or when a model falls below a target accuracy.  
- Once created, the engine returns a unique **rule ID** that you can use to track or manage this rule later.

Below is an example json, modify it accordingly.

**Function call to perform the creation of rule**

In [10]:
rule_uuid = re.create_rule(rule_json)
print("Created rule:", rule_uuid)

Created rule: 25d6a02a-d992-48f8-847b-49f118cabfaa


### 6) Retrieve a rule  

Now that we have a rule ID, we can fetch the full details of the rule from the Rules Engine.  

- This step shows the **current state** of the rule stored in the system.  
- You can use it to confirm the rule was created correctly and inspect all its parameters.  

In [12]:
rule = re.get_rule(rule_uuid)
pprint(rule)

{'Active_From': '2025-09-23T15:52:01.733788+00:00',
 'Active_To': None,
 'CI': 'OSC',
 'Data_Rules': [{'Apps': ['<TAPIS_APP_ID_1>', '<TAPIS_APP_ID_2>'],
                 'Count': 10000,
                 'Folder_Path': '/fs/ess/PAS1111/test/projectA/images',
                 'Sample_Images': True,
                 'Type': 'count',
                 'Wait_Manual': True}],
 'Rule_UUID': '25d6a02a-d992-48f8-847b-49f118cabfaa',
 'Services': ['data-label', 'model-train'],
 'TAPIS_UUID': 'molakalmuru@tacc',
 'Tapis_UserName': 'molakalmuru',
 'Type': 'data'}


### 7) List all rules  

This command retrieves all rules currently stored in the Rules Engine for the particular user

- It gives you an overview of everything you have defined so far.  
- Here we simply print the total number of rules, but you could also inspect each rule’s details if needed.  

In [24]:
rules = re.list_rules()
#pprint(rules)
print(f"Total rules: {len(rules)}")

Total rules: 11


### 8) Update an existing rule  

You can modify any part of a rule after it’s been created.  
- As an example, here we update the **CI** field of the rule.  

In [16]:
updates = {
    "CI": "TACC"
}

**Function call to update the rule**
- The function returns a confirmation.  
- We then can fetch the rule again to verify that the update was applied successfully.  

This is useful when rules need to evolve without recreating them from scratch. 

In [18]:
resp = re.update_rule(rule_uuid, updates)
pprint(resp)

# Verify
pprint(re.get_rule(rule_uuid))

{'ok': True}
{'Active_From': '2025-09-23T15:52:01.733788+00:00',
 'Active_To': None,
 'CI': 'TACC',
 'Data_Rules': [{'Apps': ['<TAPIS_APP_ID_1>', '<TAPIS_APP_ID_2>'],
                 'Count': 10000,
                 'Folder_Path': '/fs/ess/PAS1111/test/projectA/images',
                 'Sample_Images': True,
                 'Type': 'count',
                 'Wait_Manual': True}],
 'Rule_UUID': '25d6a02a-d992-48f8-847b-49f118cabfaa',
 'Services': ['data-label', 'model-train'],
 'TAPIS_UUID': 'molakalmuru@tacc',
 'Tapis_UserName': 'molakalmuru',
 'Type': 'data'}


### 9) Delete a rule  

If a rule is no longer needed, you can remove it using its unique **rule ID**.  
- Once deleted, the rule will no longer trigger any actions.  
- The API confirms the deletion by returning `{ "ok": True }`.  

This helps keep your Rules Engine clean and ensures only active, relevant rules are in place.  

In [25]:
resp = re.delete_rule(rule_uuid)
pprint(resp)

{'ok': True}
